In [1]:
import numpy as np
import torch
import torchvision
import torch.nn.functional as F
from torch.utils.data import DataLoader
import preprocessing
# import training
import importlib
import pandas as pd

importlib.reload(preprocessing)
# importlib.reload(training)

<module 'preprocessing' from 'C:\\Users\\dbsrh\\OneDrive\\Desktop\\Pytorch\\kaggle_digit_recognizer\\preprocessing.py'>

In [40]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size=3)
        self.conv2 = torch.nn.Conv2d(10, 15, kernel_size=3)
        self.fc1 = torch.nn.Linear(375, 50)
        self.fc2 = torch.nn.Linear(50, 10)
        
    def forward(self, x):
        # 28 X 28 --> 26 X 26 --> 13 X 13
        x = self.conv1(x)
        x = F.relu(F.max_pool2d(x, 2))
        # 13 X 13 --> 11 X 11 --> 5 X 5
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        # 5 X 5 X 15
        x = x.view(-1, 375)
        x = self.fc1(x)
        x = F.dropout(x, p=0.3, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [3]:
# Define CNN model
model_ft = Model()

if torch.cuda.is_available():
    model_ft.cuda()

# Define image data transforms
transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToPILImage(),
    torchvision.transforms.Resize((28, 28)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5], [0.5])
])


# Load data
valid_fraction = 0.1

csv_file = pd.read_csv('./digit-recognizer/train.csv')
csv_file = csv_file.to_numpy().astype('uint8')

boundary = int(len(csv_file) * (1 - 0.1))

train_data = preprocessing.DigitDataset(csv_file[:boundary], transforms, phase='training')
valid_data = preprocessing.DigitDataset(csv_file[boundary:], transforms, phase='training')

train_data_loader = DataLoader(
    train_data,
    batch_size = 32,
    shuffle = True,
    num_workers = 3
)

valid_data_loader = DataLoader(
    valid_data,
    batch_size = 32,
    shuffle = True,
    num_workers = 3
)

datasets = {'train' : train_data_loader, 'valid' : valid_data_loader}
datasets_size = {'train' : len(train_data_loader.dataset), 'valid' : len(valid_data_loader.dataset)}

In [4]:
optimizer_ft = torch.optim.Adam(model_ft.parameters(), lr=0.01)
scheduler_ft = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=7)

In [41]:
training(model_ft, datasets, optimizer_ft, scheduler_ft)

-------------- Epoch 0 --------------


E:\Users\dbsrh\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


train Epoch Losses : 0.2616
train Epoch Accuracy : 0.9231
valid Epoch Losses : 0.2272
valid Epoch Accuracy : 0.9348
-------------- Epoch 1 --------------
train Epoch Losses : 0.2559
train Epoch Accuracy : 0.9242
valid Epoch Losses : 0.2226
valid Epoch Accuracy : 0.9374
-------------- Epoch 2 --------------
train Epoch Losses : 0.2478
train Epoch Accuracy : 0.9279
valid Epoch Losses : 0.2159
valid Epoch Accuracy : 0.9388
-------------- Epoch 3 --------------
train Epoch Losses : 0.2421
train Epoch Accuracy : 0.9289
valid Epoch Losses : 0.2119
valid Epoch Accuracy : 0.9407
-------------- Epoch 4 --------------
train Epoch Losses : 0.2368
train Epoch Accuracy : 0.9317
valid Epoch Losses : 0.2060
valid Epoch Accuracy : 0.9414
-------------- Epoch 5 --------------
train Epoch Losses : 0.2300
train Epoch Accuracy : 0.9324
valid Epoch Losses : 0.2016
valid Epoch Accuracy : 0.9429
-------------- Epoch 6 --------------
train Epoch Losses : 0.2277
train Epoch Accuracy : 0.9346
valid Epoch Losses

([tensor(0.2616, device='cuda:0', grad_fn=<DivBackward0>),
  tensor(0.2559, device='cuda:0', grad_fn=<DivBackward0>),
  tensor(0.2478, device='cuda:0', grad_fn=<DivBackward0>),
  tensor(0.2421, device='cuda:0', grad_fn=<DivBackward0>),
  tensor(0.2368, device='cuda:0', grad_fn=<DivBackward0>),
  tensor(0.2300, device='cuda:0', grad_fn=<DivBackward0>),
  tensor(0.2277, device='cuda:0', grad_fn=<DivBackward0>),
  tensor(0.2268, device='cuda:0', grad_fn=<DivBackward0>),
  tensor(0.2250, device='cuda:0', grad_fn=<DivBackward0>),
  tensor(0.2252, device='cuda:0', grad_fn=<DivBackward0>)],
 [tensor(0.2272, device='cuda:0', grad_fn=<DivBackward0>),
  tensor(0.2226, device='cuda:0', grad_fn=<DivBackward0>),
  tensor(0.2159, device='cuda:0', grad_fn=<DivBackward0>),
  tensor(0.2119, device='cuda:0', grad_fn=<DivBackward0>),
  tensor(0.2060, device='cuda:0', grad_fn=<DivBackward0>),
  tensor(0.2016, device='cuda:0', grad_fn=<DivBackward0>),
  tensor(0.2005, device='cuda:0', grad_fn=<DivBackward0